In [1]:
import os

import torch

import numpy as np
import matplotlib.pyplot as plt

from descwl_shear_sims.galaxies import FixedGalaxyCatalog
from descwl_shear_sims.galaxies import WLDeblendGalaxyCatalog
from descwl_shear_sims.stars import StarCatalog

from descwl_shear_sims.sim import make_sim

from descwl_shear_sims.psfs import make_fixed_psf
from descwl_shear_sims.psfs import make_ps_psf

from descwl_shear_sims.sim import get_se_dim
from tqdm import tqdm

### 1. Generate images and catalogs

Draw 200 values of shear1 (g1) and shear2 (g2) between -0.02 and 0.02. So either uniform on that interval or Normal(0, $0.01^2$).

Generate 200 images of size 2048x2048 with those values of shear1 and shear2.

Will have to refer to papers to determine simulation settings.

In [2]:
# Original Setting
seed = 8312
rng = np.random.RandomState(seed)


# number of simulation trial
ntrial = 2

# size of the coadded image (in pixel)
coadd_dim = 2038

# buffer zone size (in pixel) around the image to prevent edge effects
buff = 50

for trial in range(ntrial):
    print('trial: %d/%d' % (trial+1, ntrial))

    # galaxy catalog; you can make your own
    galaxy_catalog = FixedGalaxyCatalog(
        rng=rng,
        coadd_dim=coadd_dim,
        buff=buff,
        layout='random',
        mag=25,
        hlr=1.0,
    )

    # make a constant gaussian psf
    psf = make_fixed_psf(psf_type='gauss')

    # generate some simulation data, with a particular shear
    sim_data = make_sim(
        rng=rng,
        galaxy_catalog=galaxy_catalog,
        coadd_dim=coadd_dim,
        g1=0.02,
        g2=0.00,
        bands = ['r', 'i', 'z'],
        psf=psf,
    )

trial: 1/2
trial: 2/2


In [3]:

image = torch.tensor(sim_data['band_data']['r'][0].image.array)

In [6]:
import os
import torch

# Target path
save_dir = "/data/scratch/taodingr/weak_lensing/descwl"

# Create the directory if it doesn't exist
os.makedirs(save_dir, exist_ok=True)

torch.save(image, "/data/scratch/taodingr/weak_lensing/descwl/example_image.pt")

Generate `n = 200` images and catalogs, same setting with the example, modified sizes. Sample shears from $N(0, 0.01^2)$

In [3]:
def Generate_single_img_catalog(ntrial, rng, mag, hlr, psf, morph, layout, coadd_dim, buff, sep, g1, g2, bands, noise_factor):
    """
    Generate one gatalog and image

    Input:
    ntrial: 
        number of simulation trial
    rng: np.random.RandomState
        The random number generator
    mag: float
        Magnitude of all objects. Objects brighter than magntiude 17 (e.g., 14
        since mags are opposite) tend to cause the Rubin Observatory science
        pipeline detection algorithm to misdetect isolted objects in unphysical
        ways. This effect causes the shear response to be non-linear and so
        metadetect will fail. For this reason, you should use the default
        magnitude of 17 or fainter for this kind of galaxy.
    hlr: float
        Half light radius of all objects
    psf: GSObject or PowerSpectrumPSF
        The psf object or power spectrum psf
    morph: str
        Galaxy morphology, 'exp', 'dev' or 'bd', 'bdk'.  Default 'exp'
    layout: string | Layout, optional
        The layout of objects, either 'grid' or 'random'
    coadd_dim: int, optional
        dimensions of the coadd
    buff: int, optional
        Buffer region with no objects, on all sides of image.  Ingored
        for layout 'grid'.  Default 0.
    pixel_scale: float, optional
        pixel scale in arcsec
    sep: float | None
        Separation of galaxies in arcsec
    g1: 
        shear distortions 1
    g2: 
        shear distortions 2
    bands: 
        the band
    
    Output: 
    simulated image tensor with size (num_of_bands, coadd_dim+10, coadd_dim-10)
    """
    for trial in range(ntrial):
        #print('trial: %d/%d' % (trial+1, ntrial))

        # galaxy catalog; you can make your own
        galaxy_catalog = FixedGalaxyCatalog(
            rng=rng,
            coadd_dim=coadd_dim,
            buff=buff,
            layout=layout,
            mag=mag,
            hlr=hlr,
            morph=morph
        )

        # make a constant gaussian psf
        psf = psf

        # generate some simulation data, with a particular shear
        sim_data = make_sim(
            rng=rng,
            galaxy_catalog=galaxy_catalog,
            coadd_dim=coadd_dim,
            g1=g1,
            g2=g2,
            bands = bands,
            psf=psf,
            noise_factor=noise_factor
        )

        images = []
        for band in bands:
            image_np = sim_data['band_data'][band][0].image.array
            images.append(torch.tensor(image_np, dtype=torch.float32))
            image_tensor = torch.stack(images, dim=0)  
        
        return image_tensor

In [4]:
def Generate_img_catalog(num_data, ntrial, rng, mag, hlr, psf, morph, layout, coadd_dim, buff, sep, bands, noise_factor):
    """
    Generate a number of catalogs and images

    Input:
    num_data: 
        number of images
    ntrial: 
        number of simulation trial
    rng: np.random.RandomState
        The random number generator
    mag: float
        Magnitude of all objects. Objects brighter than magntiude 17 (e.g., 14
        since mags are opposite) tend to cause the Rubin Observatory science
        pipeline detection algorithm to misdetect isolted objects in unphysical
        ways. This effect causes the shear response to be non-linear and so
        metadetect will fail. For this reason, you should use the default
        magnitude of 17 or fainter for this kind of galaxy.
    hlr: float
        Half light radius of all objects
    psf: GSObject or PowerSpectrumPSF
        The psf object or power spectrum psf
    morph: str
        Galaxy morphology, 'exp', 'dev' or 'bd', 'bdk'.  Default 'exp'
    layout: string | Layout, optional
        The layout of objects, either 'grid' or 'random'
    coadd_dim: int, optional
        dimensions of the coadd
    buff: int, optional
        Buffer region with no objects, on all sides of image.  Ingored
        for layout 'grid'.  Default 0.
    pixel_scale: float, optional
        pixel scale in arcsec
    sep: float | None
        Separation of galaxies in arcsec
    bands: 
        the band
    noise_factor: float, optional
        Factor by which to multiply the noise, default 1

    Output:
        images: images tensor (num_data, num_of_bands, coadd_dim+10, coadd_dim-10)
        catalogs: dictionary containing generated shear values for g1 and g2
    """
    # generate random shear values
    # Draw from N(0, 0.01^2)
    g1 = np.random.normal(loc=0.0, scale=0.01, size=num_data)
    g2 = np.random.normal(loc=0.0, scale=0.01, size=num_data)

    # Clip values to [-0.02, 0.02]
    g1 = np.clip(g1, -0.02, 0.02)
    g2 = np.clip(g2, -0.02, 0.02)

    psf = make_fixed_psf(psf_type='gauss')

    images_list = []
    for iter in tqdm(range(num_data)):
        single_image = Generate_single_img_catalog(ntrial, rng, mag, hlr, psf, morph, layout, coadd_dim, buff, sep, g1[iter], g2[iter], bands, noise_factor)
        images_list.append(single_image)

    catalogs = {
        "g1": g1,
        "g2": g2
    }

    images = torch.stack(images_list, dim=0)

    return images, catalogs

    

In [5]:
# Default setting for the two function 
seed = None

num_data = 200
ntrial = 2
rng = np.random.RandomState(seed)
mag = 17.0
hlr = 0.5
psf = make_fixed_psf(psf_type='gauss')
morph = 'exp'
layout = None # 'random' or 'grid'
coadd_dim = None # int
buff = 0 
sep = None 
bands = ['i']
noise_factor = 1.0 


In [6]:
# Example Setting
seed = 8312

num_data = 200
ntrial = 2
rng = np.random.RandomState(seed)
mag = 25
hlr = 1.0
psf = make_fixed_psf(psf_type='gauss')
morph = 'exp'
layout = 'random'
coadd_dim = 2038
buff = 50 
sep = None 
bands = ['r', 'i', 'z']
noise_factor = 1 


images, catalogs = Generate_img_catalog(num_data, ntrial, rng, mag, hlr, psf, morph, layout, coadd_dim, buff, sep, bands, noise_factor)

100%|██████████| 200/200 [27:27<00:00,  8.24s/it]


### 2. Save the images and catalogs as tensors

The tensor of images should be # images x # bands x 2048 x 2048.

Catalogs: As a first step, just store shear1 and shear2 in a dictionary.

In [36]:
images.shape

torch.Size([200, 3, 2048, 2048])

In [37]:
catalogs

{'g1': array([ 9.72779897e-04, -7.71915834e-03,  1.20102066e-02,  1.59860750e-02,
         5.48684738e-03,  1.23090601e-02,  1.33663626e-02, -1.71942976e-02,
         7.52089729e-04,  2.73219427e-03, -8.20979511e-03, -6.66936524e-03,
        -1.74411034e-02, -4.82805953e-03,  1.22674728e-02, -1.50582384e-03,
        -5.25710365e-03,  1.94809360e-03,  1.19324657e-02, -3.83923212e-03,
         6.72602134e-03,  1.29600399e-02,  8.53371099e-03, -3.89582676e-03,
        -1.00083489e-04,  1.19898384e-02,  1.12011870e-02,  6.80665739e-04,
         9.73616317e-04, -5.63481363e-03, -7.89842971e-03,  1.36015584e-03,
        -4.60488446e-03, -1.93125795e-03, -8.35731291e-03,  7.95011972e-03,
        -7.51028385e-03, -3.57652578e-03,  2.34070279e-03,  9.77531468e-03,
        -3.57395239e-03, -1.16387686e-02,  6.86084976e-03, -5.42430439e-03,
        -7.41670413e-03, -5.92666816e-03,  2.00000000e-02,  7.36406161e-03,
        -3.12778559e-03, -6.43615998e-03,  2.05987696e-03, -1.91242446e-03,
      

#### 3. Save the tensors to `/data/scratch/weak_lensing/descwl`

In [12]:
import os
import torch

# Target path
save_dir = "/data/scratch/weak_lensing/descwl"

# Create the directory if it doesn't exist
os.makedirs(save_dir, exist_ok=True)

# Save the tensor
torch.save(images, "/data/scratch/taodingr/weak_lensing/descwl/images.pt")
torch.save(catalogs, "/data/scratch/taodingr/weak_lensing/descwl/catalog.pt")


RuntimeError: File /data/scratch/weak_lensing/descwl/images.pt cannot be opened.

### Simulation setting in `Metadetection Weak Lensing for the Vera C. Rubin Observatory` by Sheldon et al.

- identical, round exponential, S/N = 10000

- half light radius 0.5 arcsecond galaxies
- placed in the grid layout 
- 9.5 arcsecond spacing, for a density of 40 per square arcminute
- For computation efficiency, only use `i` band with a single image wraped to coadded frame
- Fixed circular Moffat PSF with FWHM=0.8
- a cut S/N > 10 to remove spurious detections



In [9]:
seed = 42

num_data = 200
ntrial = 2
rng = np.random.RandomState(seed)
mag = 25
hlr = 0.5
psf = make_fixed_psf(psf_type='moffat', psf_fwhm=0.8)
morph = 'exp'
layout = 'grid'
coadd_dim = 2038
buff = 1
sep = 9.5
bands = ['i']
noise_factor=0.0001

images_meta, catalogs_meta = Generate_img_catalog(num_data, ntrial, rng, mag, hlr, psf, morph, layout, coadd_dim, buff, sep, bands, noise_factor)

100%|██████████| 200/200 [04:40<00:00,  1.40s/it]


In [10]:
images_meta.shape

torch.Size([200, 1, 2048, 2048])

### Simulation Settings in  `A differentiable perturbation-based weak lensing shear estimator`  by Li et al.

- Image pixel scale is set to 0.2 (default)

- round Moffat PSF with FWHM=0.8 (the author also used HSC-like PSF with ellipticity $e_1 = 0.02, e_2 = -0.02$ to test the additive bias)

- noise level matched the LSST 10 years setting (I assume default here)

- use `'griz'` bands, same galaxy profiles and PSF across bands

- noise variance are different across bands

- no dithering (default)

- coadd image with inverse variance weight of background noise

The author also used following method to contrain shear bias, not sure whether the `descwl_shear_sims` package can accommodate

- generate image pair for each galaxy with different shear ($g_1 = 0.02, g_2 = 0$) and ($g_1 = 0, g_2 = 0.02$)

- For each pair, the image have same morphology and brightness, but orthogonal to each other.

- galaxy density is about 230 spare $\text{arcmin}^{-1}$


In [7]:
seed = 42

num_data = 200
ntrial = 2
rng = np.random.RandomState(seed)
mag = 17 # default
hlr = 0.5 # default
psf = make_fixed_psf(psf_type='moffat', psf_fwhm=0.8)
morph = 'exp' # default
layout = 'random' # use example setting
coadd_dim = 2038
buff = 1
sep = None
bands = ['g', 'r', 'i', 'z']
noise_factor=1

images_adp, catalogs_adp = Generate_img_catalog(num_data, ntrial, rng, mag, hlr, psf, morph, layout, coadd_dim, buff, sep, bands, noise_factor)

100%|██████████| 200/200 [37:14<00:00, 11.17s/it]


In [8]:
images_adp.shape

torch.Size([200, 4, 2048, 2048])

### Simulation Settings in `Analytical Noise Bias Correction for Precise Weak Lensing Shear Inference` by Li et al.

- Constant shear $\gamma_1 = \pm 0.02$ (two shear or one shear?)

- The simulation is divided into 10000 subfields for each test case, and each subfield is 0.06 square degrees.

- creating a 90 degree rotated companion for each subfields

- round Moffat PSF with FWHM=0.8

- Applyed Gaussian noise after the background subtraction, to simulate the residual Poisson noise. done with package `WeakLensingDeblending`?

- standardized the images to a consistent zero point of 30

- use 'griz' bands

- anticipated ten-year LSST noise levels (I assume default here)

In [ ]:
seed = 42

num_data = 200
ntrial = 2
rng = np.random.RandomState(seed)
mag = 17 # default
hlr = 0.5 # default
psf = make_fixed_psf(psf_type='moffat', psf_fwhm=0.8)
morph = 'exp' # default
layout = 'random' # use example setting
coadd_dim = 2038
buff = 1
sep = None
bands = ['g', 'r', 'i', 'z']
noise_factor=1

images_adp, catalogs_adp = Generate_img_catalog(num_data, ntrial, rng, mag, hlr, psf, morph, layout, coadd_dim, buff, sep, bands, noise_factor)